In [21]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import math
import time

In [2]:
header = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.220 Whale/1.3.51.7 Safari/537.36',
}

In [3]:
keyword = '인공지능 문제'

In [4]:
url = 'http://search.hani.co.kr/Search?command=query&keyword=%s&media=news&submedia=&sort=d&period=all&datefrom=2016.01.01&dateto=2020.12.31&pageseq=%d'%(keyword,1)

In [5]:
res = requests.get(url, headers=header)

In [6]:
#### 페이지 수 확인
soup=BeautifulSoup(res.content, 'html.parser')

page_base=soup.find("span",{'class':'total'}).text
page_base1 = int(re.sub('[^0-9]','',page_base))
page_end=math.ceil(page_base1/10)


In [16]:
### 기사 링크 수집

href_all=[]
check_list=[] #가져오지 못한 페이지 있는지 확인하기 위해 만든 리스트

for page in range(1,page_end+1) :
    print('page :' + str(page)+ '/' + str(page_end))
    
    url = 'http://search.hani.co.kr/Search?command=query&keyword=%s&media=news&submedia=&sort=d&period=all&datefrom=2016.01.01&dateto=2020.12.31&pageseq=%d'%(keyword, page)
    
    res = requests.get(url, headers=header)  
    
    #데이터를 잘 가져오고 있는지 확인
    
    if res.status_code==200:
        check_list.append('정상')
    else:
        check_list.append('에러')
        continue
    
    ### 2페이지 이상부터 페이지 정보 안가져와졌으면 다시 가져오기

    if page>1:
        if soup==BeautifulSoup(res.content, 'html.parser'):
            while True:
                res = requests.get(url, headers=header)
                if soup!=BeautifulSoup(res.content, 'html.parser'):
                    break
    
    
     ### 기사 링크 수집
    
    href_base=[]
    
    soup=BeautifulSoup(res.content, 'html.parser')
    href = soup.find_all('dt')
    
    for i in href:
        href_base.append(i.find('a'))

    href_base=[item for item in href_base if item != None]

    for i in href_base:
        href_all.append(i['href'])
    
    ### 페이지의 데이터를 전부 가져오지 못했으면 다시 가져오기
    
    if page!=page_end:
        if len(href_base) !=10:
            while True:
                res = requests.get(url, headers=header)  

                href_base=[]
                soup=BeautifulSoup(res.content, 'html.parser')
                href = soup.find_all('dt')
 
                for i in href:
                    href_base.append(i.find('a'))

                href_base=[item for item in href_base if item != None]

                for i in href_base:
                    href_all.append(i['href'])
                    
                if len(href)==10:
                    break
    
    href=[]

page :1/116
page :2/116
page :3/116
page :4/116
page :5/116
page :6/116
page :7/116
page :8/116
page :9/116
page :10/116
page :11/116
page :12/116
page :13/116
page :14/116
page :15/116
page :16/116
page :17/116
page :18/116
page :19/116
page :20/116
page :21/116
page :22/116
page :23/116
page :24/116
page :25/116
page :26/116
page :27/116
page :28/116
page :29/116
page :30/116
page :31/116
page :32/116
page :33/116
page :34/116
page :35/116
page :36/116
page :37/116
page :38/116
page :39/116
page :40/116
page :41/116
page :42/116
page :43/116
page :44/116
page :45/116
page :46/116
page :47/116
page :48/116
page :49/116
page :50/116
page :51/116
page :52/116
page :53/116
page :54/116
page :55/116
page :56/116
page :57/116
page :58/116
page :59/116
page :60/116
page :61/116
page :62/116
page :63/116
page :64/116
page :65/116
page :66/116
page :67/116
page :68/116
page :69/116
page :70/116
page :71/116
page :72/116
page :73/116
page :74/116
page :75/116
page :76/116
page :77/116
page :78

In [17]:
## 날짜 형식대로 만드는 함수

def date_slice(date_base):
    date_base1=re.sub('[^0-9]','',date_base)
    year=date_base1[0:4]
    month = date_base1[4:6]
    day = date_base1[6:8]
    return(year,month,day)

In [18]:
### href를 url 형식으로 바꿔주기

href_url=[]
for i  in href_all:
    href_url.append('http:' + i)

In [19]:
check_list_2=[]
data_all = pd.DataFrame()
page=0

for url in href_url :
    
    page= page+1
    print('page :' + str(page) +'/' + str(len(href_url)))
    
    res = requests.get(url, headers=header)  
    
    if res.status_code==200:
        check_list_2.append('정상')
    else:
        check_list_2.append('에러')
        continue
        

    if page>1:
        if soup==BeautifulSoup(res.content, 'html.parser'):
            while True:
                res = requests.get(url, headers=header)
                if soup!=BeautifulSoup(res.content, 'html.parser'):
                    break
    
    
    
    soup=BeautifulSoup(res.content, 'html.parser')
    
    title = soup.select_one('#article_view_headline > h4').text
    
    content=soup.find('div',{'class':'text'}).text
    
    date_base=soup.find('p',{'class':'date-time'}).text
    year=date_slice(date_base)[0]
    month=date_slice(date_base)[1]
    day=date_slice(date_base)[2]
    
    data = {"Title":[title], "Content" : [content], "Year" :[year], "Month" : [month], "Day" : [day]}
    data = pd.DataFrame(data)
    
    data_all = pd.concat([data_all,data], axis=0)
    
data_all=data_all.reset_index().drop(['index'],axis=1)


page :1/1160
page :2/1160
page :3/1160
page :4/1160
page :5/1160
page :6/1160
page :7/1160
page :8/1160
page :9/1160
page :10/1160
page :11/1160
page :12/1160
page :13/1160
page :14/1160
page :15/1160
page :16/1160
page :17/1160
page :18/1160
page :19/1160
page :20/1160
page :21/1160
page :22/1160
page :23/1160
page :24/1160
page :25/1160
page :26/1160
page :27/1160
page :28/1160
page :29/1160
page :30/1160
page :31/1160
page :32/1160
page :33/1160
page :34/1160
page :35/1160
page :36/1160
page :37/1160
page :38/1160
page :39/1160
page :40/1160
page :41/1160
page :42/1160
page :43/1160
page :44/1160
page :45/1160
page :46/1160
page :47/1160
page :48/1160
page :49/1160
page :50/1160
page :51/1160
page :52/1160
page :53/1160
page :54/1160
page :55/1160
page :56/1160
page :57/1160
page :58/1160
page :59/1160
page :60/1160
page :61/1160
page :62/1160
page :63/1160
page :64/1160
page :65/1160
page :66/1160
page :67/1160
page :68/1160
page :69/1160
page :70/1160
page :71/1160
page :72/1160
p

page :555/1160
page :556/1160
page :557/1160
page :558/1160
page :559/1160
page :560/1160
page :561/1160
page :562/1160
page :563/1160
page :564/1160
page :565/1160
page :566/1160
page :567/1160
page :568/1160
page :569/1160
page :570/1160
page :571/1160
page :572/1160
page :573/1160
page :574/1160
page :575/1160
page :576/1160
page :577/1160
page :578/1160
page :579/1160
page :580/1160
page :581/1160
page :582/1160
page :583/1160
page :584/1160
page :585/1160
page :586/1160
page :587/1160
page :588/1160
page :589/1160
page :590/1160
page :591/1160
page :592/1160
page :593/1160
page :594/1160
page :595/1160
page :596/1160
page :597/1160
page :598/1160
page :599/1160
page :600/1160
page :601/1160
page :602/1160
page :603/1160
page :604/1160
page :605/1160
page :606/1160
page :607/1160
page :608/1160
page :609/1160
page :610/1160
page :611/1160
page :612/1160
page :613/1160
page :614/1160
page :615/1160
page :616/1160
page :617/1160
page :618/1160
page :619/1160
page :620/1160
page :621/

page :1096/1160
page :1097/1160
page :1098/1160
page :1099/1160
page :1100/1160
page :1101/1160
page :1102/1160
page :1103/1160
page :1104/1160
page :1105/1160
page :1106/1160
page :1107/1160
page :1108/1160
page :1109/1160
page :1110/1160
page :1111/1160
page :1112/1160
page :1113/1160
page :1114/1160
page :1115/1160
page :1116/1160
page :1117/1160
page :1118/1160
page :1119/1160
page :1120/1160
page :1121/1160
page :1122/1160
page :1123/1160
page :1124/1160
page :1125/1160
page :1126/1160
page :1127/1160
page :1128/1160
page :1129/1160
page :1130/1160
page :1131/1160
page :1132/1160
page :1133/1160
page :1134/1160
page :1135/1160
page :1136/1160
page :1137/1160
page :1138/1160
page :1139/1160
page :1140/1160
page :1141/1160
page :1142/1160
page :1143/1160
page :1144/1160
page :1145/1160
page :1146/1160
page :1147/1160
page :1148/1160
page :1149/1160
page :1150/1160
page :1151/1160
page :1152/1160
page :1153/1160
page :1154/1160
page :1155/1160
page :1156/1160
page :1157/1160
page :11

In [12]:
df = data_all

In [13]:
#불용어 처리
df['Title'] = df['Title'].str.replace(r'\[([^)]+)\]','') 
df['Title'] = df['Title'].str.replace('‘', '')
df['Title'] = df['Title'].str.replace('’', '')
df['Title'] = df['Title'].str.replace(',', '')
df['Title'] = df['Title'].str.replace('[', '')
df['Title'] = df['Title'].str.replace(']', '')
df['Title'] = df['Title'].str.replace('…', '')
df['Title'] = df['Title'].str.replace('-', '')
df['Title'] = df['Title'].str.replace('·', '')
df['Title'] = df['Title'].str.replace('“', '')
df['Title'] = df['Title'].str.replace('”', '')
df['Title'] = df['Title'].str.replace('"', '')
df['Title'] = df['Title'].str.replace('···', '')
df['Title'] = df['Title'].str.replace("'", '')
df['Title'] = df['Title'].str.replace('?', '')  

In [14]:
df['Content'] = df['Content'].str.replace(r'\(([^)]+)\)','')

In [15]:
#불용어 처리
df['Content'] = df['Content'].str.replace('[', '')
df['Content'] = df['Content'].str.replace(']', '')
df['Content'] = df['Content'].str.replace("'", "")  
df['Content'] = df['Content'].str.replace('"', '')
df['Content'] = df['Content'].str.replace(',', '')
df['Content'] = df['Content'].str.replace(r'\\n', '') 
df['Content'] = df['Content'].str.replace(r'\\r', '')
df['Content'] = df['Content'].str.replace(r'\\xa0', '')
df['Content'] = df['Content'].str.replace('‘', '')
df['Content'] = df['Content'].str.replace('’', '')
df['Content'] = df['Content'].str.replace('“', '')
df['Content'] = df['Content'].str.replace('”', '')
df['Content'] = df['Content'].str.replace('.', '')
df['Content'] = df['Content'].str.replace('//', '')
df['Content'] = df['Content'].str.replace('ㆍ', '')
df['Content'] = df['Content'].str.replace('  ', '')
df['Content'] = df['Content'].str.replace('·', '')
df['Content'] = df['Content'].str.replace('…', '')
df['Content'] = df['Content'].str.replace('〈br〉', '')
df['Content'] = df['Content'].str.replace('「', '')
df['Content'] = df['Content'].str.replace('」', '')
df['Content'] = df['Content'].str.replace(r'\\', '')
df['Content'] = df['Content'].str.replace(r'/*', '')
df['Content'] = df['Content'].str.replace('◇','')
df['Content'] = df['Content'].str.replace('＜','')
df['Content'] = df['Content'].str.replace('＞','')
df['Content'] = df['Content'].str.replace('△','')
df['Content'] = df['Content'].str.replace('▲','')
df['Content'] = df['Content'].str.replace('=','')
df['Content'] = df['Content'].str.replace('||','')
df['Content'] = df['Content'].str.replace('{','')
df['Content'] = df['Content'].str.replace('}','')
df['Content'] = df['Content'].str.replace('()','')
df['Content'] = df['Content'].str.replace(';','')
df['Content'] = df['Content'].str.replace(')','')
df['Content'] = df['Content'].str.replace('(','')
df['Content'] = df['Content'].str.replace('*','')
df['Content'] = df['Content'].str.replace('＜1-2＞','')
df['Content'] = df['Content'].str.replace('\s{2,}','') 
df['Content'] = df['Content'].str.replace('#\S+','') #delete#tag  
df['Content'] = df['Content'].str.replace('@\S+','') #@xx제거
df['Content'] = df['Content'].str.replace('주요기사.*','') 
df['Content'] = df['Content'].str.replace('©','') 
df['Content'] = df['Content'].str.replace('○',' ')
df['Content'] = df['Content'].str.replace('■','')
df['Content'] = df['Content'].str.replace('●','')
df['Content'] = df['Content'].str.replace('▽','')
df['Content'] = df['Content'].str.replace('《','')
df['Content'] = df['Content'].str.replace('》','')
df['Content'] = df['Content'].str.replace('〃','')
df['Content'] = df['Content'].str.replace(r'ufeff','') 
df['Content'] = df['Content'].str.replace('사진 게티이미지','')

df['Content'] = df['Content'].str.lstrip() #공백제거

In [16]:
df.insert(5, "URL",href_url, True)

In [22]:
nows = time.localtime()
times = str(nows.tm_year) + '-' + str(nows.tm_mon) + '-' + str(nows.tm_mday)
name = times + '한겨례.xlsx'

writer = pd.ExcelWriter(name, engine='xlsxwriter', options={'strings_to_urls': True})
df.to_excel(writer)
writer.close()

NameError: name 'df' is not defined